# Invoke a SageMaker Endpoint from Kinesis Firehose (Data Transformation)

<img src="img/kinesis_firehose_transform.png" width="90%" align="left">

In [ ]:
import boto3
import sagemaker
import pandas as pd
import json

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
firehose = boto3.Session().client(service_name='firehose', region_name=region)
lam = boto3.Session().client(service_name='lambda', region_name=region)

## Check IAM Roles Are In Place

In [ ]:
%store -r iam_lambda_role_name

In [ ]:
try:
    iam_lambda_role_name
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(iam_lambda_role_name)

In [ ]:
%store -r iam_lambda_role_passed

In [ ]:
try:
    iam_lambda_role_passed
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(iam_lambda_role_passed)

In [ ]:
if not iam_lambda_role_passed:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')
else:
    print('[OK]')

In [ ]:
%store -r iam_role_lambda_arn

In [ ]:
try:
    iam_role_lambda_arn
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(iam_role_lambda_arn)

## Review Lambda Function

In [ ]:
lambda_fn_name_invoke_ep='InvokeSageMakerEndpointFromKinesis'

In [ ]:
%store lambda_fn_name_invoke_ep

In [ ]:
!pygmentize src/invoke_sm_endpoint_from_kinesis.py

## Zip The Function Code

In [ ]:
!zip src/InvokeSageMakerEndpointFromKinesis.zip src/invoke_sm_endpoint_from_kinesis.py

## Load the .zip File as Binary Code

In [ ]:
with open('src/InvokeSageMakerEndpointFromKinesis.zip', 'rb') as f: 
    code = f.read()

## Create The Lambda Function

See for Python 3.x support:   
https://stackoverflow.com/questions/54407298/why-am-i-getting-a-runtime-marshalerror-when-using-this-code-in-zapier

In [ ]:
from botocore.exceptions import ClientError

try: 
    response = lam.create_function(
        FunctionName='{}'.format(lambda_fn_name_invoke_ep),
        Runtime='python3.7',
        Role='{}'.format(iam_role_lambda_arn),
        Handler='src/invoke_sm_endpoint_from_kinesis.lambda_handler',
        Code={
            'ZipFile': code
        },
        Description='Query SageMaker Endpoint for star rating prediction on reveiw input text.',
        Timeout=900,
        MemorySize=128,
        Publish=True
    )
    print('Lambda Function {} successfully created.'.format(lambda_fn_name_invoke_ep))

except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceConflictException':
        print('Lambda Function {} already exists. This is OK.'.format(lambda_fn_name_invoke_ep))
    else:
        print('Error: {}'.format(e))

In [ ]:
response = lam.get_function(FunctionName=lambda_fn_name_invoke_ep)
print(json.dumps(response, indent=4, sort_keys=True, default=str))

In [ ]:
response = lam.get_function(FunctionName=lambda_fn_name_invoke_ep)

lambda_fn_arn_invoke_ep = response['Configuration']['FunctionArn']
print(lambda_fn_arn_invoke_ep)

In [ ]:
%store lambda_fn_arn_invoke_ep

## Review Lambda Function

In [ ]:
from IPython.core.display import display, HTML
        
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/lambda/home?region={}#/functions/{}"> Lambda Function</a></b>'.format(region, lambda_fn_name_invoke_ep)))


## Check SageMaker Endpoint 

In [ ]:
%store -r pytorch_model_name

In [ ]:
try:
    pytorch_model_name
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(pytorch_model_name)

In [ ]:
try:
    waiter = sm.get_waiter('endpoint_in_service')
    waiter.wait(EndpointName=pytorch_model_name)
except:
    print('###################')
    print('The endpoint is not running.')
    print('Please re-run the model deployment section to deploy the endpoint.')
    print('###################')  

## Test Model Predictions

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

model = Predictor(endpoint_name=pytorch_model_name,
                  sagemaker_session=sess)

In [ ]:
review = 'This is great'

model_predicted_class = model.predict(review).decode('utf-8')

print('[Predicted Star Rating: {}]'.format(model_predicted_class), review)

## Configure Lambda With Endpoint

In [ ]:
response = lam.update_function_configuration(
            FunctionName=lambda_fn_name_invoke_ep,
            Environment={
                'Variables': {
                    'ENDPOINT_NAME': pytorch_model_name
                }
            }
        )

# Store Variables for Next Notebooks

In [ ]:
%store

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();